# Advances in Scientific and Applied Accounting – ASAA

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all('h3', {'class': 'title'})
    a_links = []
    for item in results:
        try:
            remover_links = 'editorial' in item.find('a').text.lower()
            remover_links = remover_links or 'folha de rosto' in item.find('a').text.lower()
            remover_links = remover_links or 'dados da edição ' in item.find('a').text.lower()
            if not remover_links:
                a_links.append(item.find('a')['href'])
        except:
            pass
    return a_links

In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links

In [5]:
url_search_text = 'https://asaa.anpcont.org.br/index.php/asaa/search/index?query=__query__&searchJournal=1&authors=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&searchPage=__pg_num__#results'

links = listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)

link = links[0]

link


'https://asaa.anpcont.org.br/index.php/asaa/article/view/500'

In [6]:
links

['https://asaa.anpcont.org.br/index.php/asaa/article/view/500',
 'https://asaa.anpcont.org.br/index.php/asaa/article/view/590',
 'https://asaa.anpcont.org.br/index.php/asaa/article/view/508',
 'https://asaa.anpcont.org.br/index.php/asaa/article/view/569',
 'https://asaa.anpcont.org.br/index.php/asaa/article/view/627',
 'https://asaa.anpcont.org.br/index.php/asaa/article/view/415',
 'https://asaa.anpcont.org.br/index.php/asaa/article/view/632',
 'https://asaa.anpcont.org.br/index.php/asaa/article/view/410',
 'https://asaa.anpcont.org.br/index.php/asaa/article/view/528',
 'https://asaa.anpcont.org.br/index.php/asaa/article/view/426',
 'https://asaa.anpcont.org.br/index.php/asaa/article/view/636',
 'https://asaa.anpcont.org.br/index.php/asaa/article/view/698',
 'https://asaa.anpcont.org.br/index.php/asaa/article/view/838',
 'https://asaa.anpcont.org.br/index.php/asaa/article/view/684',
 'https://asaa.anpcont.org.br/index.php/asaa/article/view/525',
 'https://asaa.anpcont.org.br/index.php/

In [7]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [9]:
def limpar_afiliacao(afiliacao):
    afiliacao = ' '.join(afiliacao.split())
    afiliacao = afiliacao.split(',')[0]
    afiliacao = afiliacao.split(' - ')[0]
    afiliacao = afiliacao.split('(')[0]
    afiliacao = afiliacao.split('/')[0]
    return afiliacao.strip()

In [10]:
def captura_titulo(soup):
    try:
        title = soup.find('h1', {'class': 'page_title'}).text
        title = ' '.join(title.split())
    except:
        title = ''
    return {'Título': title}

In [11]:
captura_titulo(soup)

{'Título': 'RESPONSABILIDADE AMBIENTAL, CUSTO DE CAPITAL, RISCO E ENDIVIDAMENTO'}

In [12]:
def captura_data(soup):
    try:
        date_tag = soup.find('div', {'class': 'item published'})
        date = date_tag.find('div', {'class': 'value'}).text
        date = ' '.join(date.split())
    except:
        date = ''
    return {'Data de Publicação': date[:10]}

In [13]:
captura_data(soup)

{'Data de Publicação': '18/11/2019'}

In [14]:
def captura_resumo(soup):
    try:
        abstract_section = soup.find('section', {'class': 'item abstract'})
        abstract = abstract_section.text
        abstract = abstract.replace('Abstract:', '')
        abstract = abstract.replace('Abstract', '').strip()
        #abstract = ' '.join(abstract.split())
    except:
        abstract = ''
    return {'Resumo': abstract}

In [15]:
captura_resumo(soup)

{'Resumo': 'O presente estudo teve por objetivo verificar o efeito da sinalização ambiental na estrutura de capital. A certificação ISO 14001 foi empregada como proxy de sinalização para o mercado de empresas que são comprometidas com sustentabilidade ambiental. Os dados coletados se referem ao período de 1995 a 2016, sendo a amostra final de 2365 firmas/ano. Utilizou-se um modelo “difference in difference” com efeito fixo para empresa, com o intuito de se estimar o efeito da obtenção da certificação na estrutura de capital, risco e nível de endividamento mediante a comparação de empresas que receberam a respectiva certificação do Sistema de Gestão Ambiental com empresas que não possuem tal certificação. Um Propensity Score Matching também foi efetuado para se mitigar um possível viés de auto-seleção. Os resultados sugerem que embora a adoção de práticas de gerenciamento exigidas pelos padrões ISO14001 não reduza em média o risco sistêmico das empresas, percebe-se alteração na estrutur

In [16]:
def captura_palavras_chave(soup):
    try:
        kw_section = soup.find('section', {'class': 'item keywords'})
        kw = kw_section.text.replace('Keywords:', '')
        kw = ' '.join(kw.split())
        kw = kw.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    if kw == '':
        try:
            kw_section = soup.find('section', {'class': 'item abstract'})
            p_list = kw_section.find_all('p')
            for p in p_list:
                if 'Palavras-chave' in p.text:
                    kw_full = p.text
                    kw = kw_full.replace('Palavras-chave', '').replace(':', '')
                    kw = ' '.join(kw.split())
                    kw = kw.replace(',', '.').replace(';', '.')
        except:
            pass
    return {'Palavras-chave': kw}

In [17]:
captura_palavras_chave(soup)

{'Palavras-chave': 'Certificação. Sinalização. Custo de capital. Risco. Dívida.'}

In [18]:
def captura_autores(soup):
    autores = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        autores = authors_ul.find_all('span', {'class': 'name'})
        autores = [' '.join(a.text.split()) for a in autores]
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [19]:
captura_autores(soup)

{'Qtd. de Autores': 4,
 'Autor 1': 'Jessica Prado Martins',
 'Autor 2': 'Danilo Soares Monte-mor',
 'Autor 3': 'Silvania Neris Nossa',
 'Autor 4': 'Valcemiro Nossa',
 'Autor 5': '',
 'Autor 6': ''}

In [20]:
def captura_afiliacao(soup):
    afiliacao = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        afiliacao = authors_ul.find_all('span', {'class': 'affiliation'})
        afiliacao = [' '.join(a.text.split()) for a in afiliacao]
        afiliacao = [limpar_afiliacao(a) for a in afiliacao]
    except:
        pass
    afiliacao = completar_lista(afiliacao)
    
    key_affiliation = [f"Afiliação {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, afiliacao))

In [21]:
captura_afiliacao(soup)

{'Afiliação 1': 'Fucape Business School',
 'Afiliação 2': 'Fucape Business School',
 'Afiliação 3': 'Fucape Business School',
 'Afiliação 4': 'Fucape Business School',
 'Afiliação 5': '',
 'Afiliação 6': ''}

In [22]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Advances in Scientific and Applied Accounting – ASAA'}
    url_search_text = 'https://asaa.anpcont.org.br/index.php/asaa/search/index?query=__query__&searchJournal=1&authors=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%d/%m/%Y')
    return df

## Pesquisa por artigos

In [23]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head(3)

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2019-11-18,"RESPONSABILIDADE AMBIENTAL, CUSTO DE CAPITAL, ...",O presente estudo teve por objetivo verificar ...,Certificação. Sinalização. Custo de capital. R...,4,Jessica Prado Martins,Danilo Soares Monte-mor,Silvania Neris Nossa,Valcemiro Nossa,,,Fucape Business School,Fucape Business School,Fucape Business School,Fucape Business School,,,https://asaa.anpcont.org.br/index.php/asaa/art...,Advances in Scientific and Applied Accounting ...
1,2020-08-31,DESPESAS DE EXERCÍCIOS ANTERIORES: UMA ANÁLISE...,O registro contábil de Despesas de Exercícios ...,Despesas de Exercícios Anteriores. Planejament...,2,Saulo Silva Lima Filho,Blênio Cezar Severo Peixe,,,,,Universidade Federal do Paraná,Universidade Federal do Paraná Departamento de...,,,,,https://asaa.anpcont.org.br/index.php/asaa/art...,Advances in Scientific and Applied Accounting ...
2,2020-01-23,A CERTIFICAÇÃO ISO 14001 E O ENDIVIDAMENTO DAS...,A certificação ISO 14001 compreende um conjunt...,Certificação. ISO 14001. Endividamento. B3. Co...,4,Enezio Mariano da Costa,Valcemiro Nossa,Silvania Neris Nossa,Vania Maria da Costa Borgerth,,,Fucape Business School,Fucape Business School,Fucape Business School,Fucape Business School,,,https://asaa.anpcont.org.br/index.php/asaa/art...,Advances in Scientific and Applied Accounting ...


In [24]:
df.to_csv('../data/ASAA_ANPCONT.csv', index=False)

In [25]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2019-11-18,"RESPONSABILIDADE AMBIENTAL, CUSTO DE CAPITAL, ...",O presente estudo teve por objetivo verificar ...,Certificação. Sinalização. Custo de capital. R...,4,Jessica Prado Martins,Danilo Soares Monte-mor,Silvania Neris Nossa,Valcemiro Nossa,,,Fucape Business School,Fucape Business School,Fucape Business School,Fucape Business School,,,https://asaa.anpcont.org.br/index.php/asaa/art...,Advances in Scientific and Applied Accounting ...
1,2020-08-31,DESPESAS DE EXERCÍCIOS ANTERIORES: UMA ANÁLISE...,O registro contábil de Despesas de Exercícios ...,Despesas de Exercícios Anteriores. Planejament...,2,Saulo Silva Lima Filho,Blênio Cezar Severo Peixe,,,,,Universidade Federal do Paraná,Universidade Federal do Paraná Departamento de...,,,,,https://asaa.anpcont.org.br/index.php/asaa/art...,Advances in Scientific and Applied Accounting ...
2,2020-01-23,A CERTIFICAÇÃO ISO 14001 E O ENDIVIDAMENTO DAS...,A certificação ISO 14001 compreende um conjunt...,Certificação. ISO 14001. Endividamento. B3. Co...,4,Enezio Mariano da Costa,Valcemiro Nossa,Silvania Neris Nossa,Vania Maria da Costa Borgerth,,,Fucape Business School,Fucape Business School,Fucape Business School,Fucape Business School,,,https://asaa.anpcont.org.br/index.php/asaa/art...,Advances in Scientific and Applied Accounting ...
3,2019-11-18,A INFLUÊNCIA DA ESTRUTURAÇÃO NA RELAÇÃO ENTRE ...,O tema da pesquisa versa sobre a estruturação ...,Sistemas de Controle Gerencial. Alavancas de C...,4,Ivan Rafael Defaveri,Delci Grapegia Dal Vesco,Carlos Alberto Diehl,Leandro Augusto Toigo,,,Universidade Estadual do Oeste do Paraná,Universidade Estadual do Oeste do Paraná,Universidade Vale do Rio dos Sinos,Universidade Estadual do Oeste do Paraná,,,https://asaa.anpcont.org.br/index.php/asaa/art...,Advances in Scientific and Applied Accounting ...
4,2020-04-22,CUSTOS DA QUALIDADE EM HOSPITAIS ACREDITADOS,Esse estudo tem como objetivo identificar como...,Custos da qualidade. Custos de controle. Custo...,4,Katia Abbas,Paulo Henrique Rezende da Silva,Marcia Maria Dos Santos Bortolocci Espejo,Leticia Matiolli Grejo,,,Universidade Estadual de Maringá,Possui graduação em Ciências Contábeis pela Un...,Universidade Federal do Mato Grosso do SUL,Universidade Estadual do Paraná,,,https://asaa.anpcont.org.br/index.php/asaa/art...,Advances in Scientific and Applied Accounting ...
5,2019-07-23,CARACTERÍSTICAS INSTITUCIONAIS DOS PAÍSES E PR...,Esta pesquisa objetiva investigar a relação en...,Passivos Ambientais. Provisões Ambientais. Pas...,3,Thiago Alberto dos Reis Prado,Maísa de Souza Ribeiro,Marcelo Botelho de Moraes,,,,Universidade Federal de Uberlândia,Universidade de São Paulo,USP,,,,https://asaa.anpcont.org.br/index.php/asaa/art...,Advances in Scientific and Applied Accounting ...
6,2020-04-22,PARTICIPAÇÃO E CONSTITUIÇÃO DE RESERVAS ORÇAME...,Este estudo tem como objetivo verificar em que...,Participação orçamentária. Incerteza ambiental...,3,Danrlei Anderson Peyerl,Franciele Beck,Daniel Magalhães Mucci,,,,Universidade Regional de Blumenau,Universidade Regional de Blumenau,Universidade de São Paulo,,,,https://asaa.anpcont.org.br/index.php/asaa/art...,Advances in Scientific and Applied Accounting ...
7,2019-11-18,A PRECARIEDADE DO CONTEÚDO INFORMACIONAL DOS A...,A pesquisa analisou o conteúdo informacional d...,Riscos Fiscais. Lei de Responsabilidade Fiscal...,4,Ricardo Rocha de Azevedo,André Carlos Busanelli de Aquino,André Feliciano Lino,Gabriel Cavalmoretti,,,Universidade Federal de Uberlândia,Universidade de São Paulo,Universidade Federal do Pará,Universidade de São Paulo,,,https://asaa.anpcont.org.br/index.php/asaa/art...,Advances in Scientific and Applied Accounting ...
8,2019-07-23,CONSOLIDAÇÃO DO MODELO DAS ALAVANCAS DE CONTRO...,Neste estudo analisa-se a consolidação do mode...,Sistemas